# A Sense of Place: Quick and Dirty Accessibility Analysis Using Open Source Network Analysis Tools	

How many residents live within cycling distance of the planned community center? What are the economic characteristics of the neighbors within walking distance of this library? How many more jobs can a person reach in 30 minutes if we put a bikeshare station at this bus stop? It used to be that answering location intelligence questions like these was expensive and time consuming, but thanks to a suite of open source software and open data, you can answer these and countless other questions that require analysis of complex geospatial networks for free in minutes. In this presentation we'll demonstrate how to use the r5r package for the popular R programming language to build a multi-modal transportation network for Central Ohio from scratch and use it to model some impacts of a large hypothetical industrial development in Licking County.

## Configure environment

In [1]:
# allocate RAM memory to Java
options(java.parameters = "-Xmx8G")

rJava::.jinit()
rJava::.jcall("java.lang.System", "S", "getProperty", "java.version")

#install.packages("r5r")
#install.packages("osmextract")
#install.packages("tidycensus")
#install.packages("geojsonio")
#install.packages("SpatialPosition")
#install.packages("httr")

library(r5r)
library(osmextract)
library(sf)
library(data.table)
library(dplyr)
#library(geojsonio)
#library(SpatialPosition)
#library(httr)

departureTime = as.POSIXct("2022-09-12 17:00:00",
                                 format = "%Y-%m-%d %H:%M:%S")

[1] "11"

Warning message:
"package 'r5r' was built under R version 4.1.3"
Loading required namespace: sf

Please make sure you have already allocated some memory to Java by running:
  options(java.parameters = '-Xmx2G').
Currently, Java memory is set to -Xmx8G

Warning message:
"package 'osmextract' was built under R version 4.1.3"
Data (c) OpenStreetMap contributors, ODbL 1.0. https://www.openstreetmap.org/copyright.
Check the package website, https://docs.ropensci.org/osmextract/, for more details.

Warning message:
"package 'sf' was built under R version 4.1.3"
Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 7.2.1; sf_use_s2() is TRUE

Warning message:
"package 'data.table' was built under R version 4.1.3"
Warning message:
"package 'dplyr' was built under R version 4.1.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 

## Prepare transportation network

In [16]:
start_time = Sys.time()
r5r_core <- setup_r5(data_path = "./input_data", verbose=TRUE)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))


Using cached network.dat from ./input_data/network.dat



[1] "Compute time: 0.235051282246908 min"


## Prepare network endpoints

In [17]:
site <- st_read("./input_data/site/site.shp")
site <- st_transform(site, crs="epsg:4326")

Reading layer `site' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\input_data\site\site.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -82.69665 ymin: 40.12325 xmax: -82.69665 ymax: 40.12325
Geodetic CRS:  WGS 84


In [18]:
park_and_ride <- st_read("./temp_data/park_and_ride.shp")
park_and_ride <- st_transform(park_and_ride, crs="epsg:4326")

Reading layer `park_and_ride' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\park_and_ride.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -82.78828 ymin: 40.0907 xmax: -82.78828 ymax: 40.0907
Geodetic CRS:  WGS 84


In [19]:
points <- st_read("./temp_data/points.shp")
points <- st_transform(points, crs="epsg:4326")

Reading layer `points' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\points.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 7800 features and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -84.01784 ymin: 39.16179 xmax: -82.03087 ymax: 40.71409
Geodetic CRS:  WGS 84


In [20]:
tract_points <- st_read("./temp_data/tract_points.shp")
tract_points <- st_transform(tract_points, crs="epsg:4326")

Reading layer `tract_points' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\tract_points.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 849 features and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -84.15688 ymin: 39.00131 xmax: -81.97585 ymax: 40.82609
Geodetic CRS:  WGS 84


## Compute travel time matrices

### Evening weekday commute, car only, outbound, neighborhoods

In [21]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = tract_points,
                          mode = c("CAR"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_CAR_outbound_neighborhoods.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."


[1] "Compute time: 0.193964835007985 min"


### Evening weekday commute, car only, outbound, whole region

In [22]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = points,
                          mode = c("CAR"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_CAR_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.102049001057943 min"


### Evening weekday commute, walk+transit, outbound, whole region

In [28]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = points,
                          mode = c("WALK", "TRANSIT"),
                          mode_egress = c("WALK"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_WALK-TRANSIT-WALK_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0203715840975444 min"


### Evening weekday commute, bike+transit, outbound, whole region

In [29]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = points,
                          mode = c("BICYCLE","TRANSIT"),
                          mode_egress = c("BICYCLE"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_BIKE-TRANSIT-BIKE_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0115390658378601 min"


### Evening weekday commute, shuttle+transit, outbound, whole region

In [30]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = park_and_ride,
                          destinations = points,
                          mode = c("WALK","TRANSIT"),
                          mode_egress = c("WALK"),
                          departure_datetime = departureTime,
                          max_trip_duration = 80)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_SHUTTLE-TRANSIT-WALK_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0114726821581523 min"


### Evening weekday commute, bus only, outbound, neighborhoods

In [23]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = tract_points,
                          mode = c("TRANSIT", "WALK"),
                          departure_datetime = departureTime,
                          max_trip_duration = 120,
                          mode_egress=c("WALK"),
                          breakdown=TRUE
                          )
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_WALK_TRANSIT_WALK_outbound_120.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."


[1] "Compute time: 0.00178538163503011 min"


### Evening weekday commute, bus + bike, outbound, neighborhoods

In [25]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = tract_points,
                          mode = c("TRANSIT", "WALK", "BICYCLE"),
                          departure_datetime = as.POSIXct("2022-09-12 16:20:00",
                                 format = "%Y-%m-%d %H:%M:%S"),
                          max_trip_duration = 120,
                          mode_egress=c("WALK"),
                          breakdown=TRUE
                          )
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_BICYCLE_TRANSIT_WALK_outbound_120.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."


[1] "Compute time: 0.00322253306706746 min"


### Evening weekday commute, bus from park and ride + implied shuttle, outbound, neighborhoods

In [24]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = park_and_ride,
                          destinations = tract_points,
                          mode = c("TRANSIT", "WALK"),
                          departure_datetime = departureTime,
                          max_trip_duration = 110,
                          mode_egress=c("WALK"),
                          breakdown=TRUE
                          )
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_SHUTTLE_TRANSIT_WALK_outbound_120_20220905170000.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."


[1] "Compute time: 0.00533106724421183 min"


## Compute detailed itineraries

### Car

In [22]:
start_time = Sys.time()
itineraries = detailed_itineraries(r5r_core = r5r_core,
  origins = site,
  destinations = tract_points,
  mode = c("CAR"),
  departure_datetime = departureTime,
  max_trip_duration = 120
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
st_write(itineraries, "./output_data/itineraries_CAR_outbound_120.shp", append=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."
Origins dataframe expanded to match the number of destinations.



[1] "Compute time: 0.164655117193858 min"


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Writing layer `itineraries_CAR_outbound_120' to data source 
  `./output_data/itineraries_CAR_outbound_120.shp' using driver `ESRI Shapefile'
Writing 523 features with 14 fields and geometry type Line String.


### Shuttle + transit + walk

In [37]:
start_time = Sys.time()
itineraries = detailed_itineraries(r5r_core = r5r_core,
  origins = site,
  destinations = park_and_ride,
  mode = c("CAR"),
  departure_datetime = as.POSIXct("2022-09-12 16:45:00",
                                 format = "%Y-%m-%d %H:%M:%S"),,
  max_trip_duration = 15
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
st_write(itineraries, "./output_data/itineraries_SHUTTLE_firstmile.shp", append=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.00289353529612223 min"


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Writing layer `itineraries_SHUTTLE_firstmile' to data source 
  `./output_data/itineraries_SHUTTLE_firstmile.shp' using driver `ESRI Shapefile'
Writing 1 features with 14 fields and geometry type Line String.


In [38]:
start_time = Sys.time()
itineraries = detailed_itineraries(r5r_core = r5r_core,
  origins = site,
  destinations = park_and_ride,
  mode = c("WALK"),
  departure_datetime = as.POSIXct("2022-09-12 16:20:00",
                                 format = "%Y-%m-%d %H:%M:%S"),,
  max_trip_duration = 40,
  walk_speed = 24
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
st_write(itineraries, "./output_data/itineraries_BIKE_firstmile.shp", append=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0014953335126241 min"


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Writing layer `itineraries_BIKE_firstmile' to data source 
  `./output_data/itineraries_BIKE_firstmile.shp' using driver `ESRI Shapefile'
Writing 1 features with 14 fields and geometry type Line String.


In [10]:
start_time = Sys.time()
itineraries = detailed_itineraries(r5r_core = r5r_core,
  origins = park_and_ride,
  destinations = tract_points,
  mode = c("TRANSIT", "WALK"),
  mode_egress = "WALK",
  departure_datetime = departureTime,
  max_trip_duration = 110
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
st_write(itineraries, "./output_data/itineraries_SHUTTLE_TRANSIT_WALK_outbound_120.shp", append=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."
Origins dataframe expanded to match the number of destinations.



[1] "Compute time: 0.303364634513855 min"


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Deleting layer `itineraries_SHUTTLE_TRANSIT_WALK_outbound_120' using driver `ESRI Shapefile'
Writing layer `itineraries_SHUTTLE_TRANSIT_WALK_outbound_120' to data source 
  `./output_data/itineraries_SHUTTLE_TRANSIT_WALK_outbound_120.shp' using driver `ESRI Shapefile'
Writing 977 features with 14 fields and geometry type Line String.


In [44]:
start_time = Sys.time()
itineraries = detailed_itineraries(r5r_core = r5r_core,
  origins = site,
  destinations = tract_points[325, ],
  mode = c("TRANSIT", "WALK", "BICYCLE", "CAR"),
  mode_egress = "WALK",
  departure_datetime = departureTime,
  max_trip_duration = 150,
  bike_speed = 24,
  shortest_path = FALSE
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
st_write(itineraries, "./output_data/itineraries_ANY_toconference_150.shp", append=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0122244993845622 min"


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Deleting layer `itineraries_ANY_toconference_150' using driver `ESRI Shapefile'
Writing layer `itineraries_ANY_toconference_150' to data source 
  `./output_data/itineraries_ANY_toconference_150.shp' using driver `ESRI Shapefile'
Writing 13 features with 14 fields and geometry type Line String.


## Clean up memory

In [ ]:
stop_r5(r5r_core)
rJava::.jgc(R.gc = TRUE)

# References

  1. [r5r: Rapid Realistic Routing with R5 in R](https://ipeagit.github.io/r5r/) (Project website)
  1. [Intro to r5r: Rapid Realistic Routing with R5 in R](https://ipeagit.github.io/r5r/articles/r5r.html) (Vignette with multi-modal routing example using sample data)
  1. [r5r: Rapid Realistic Routing on Multimodal Transport Networks with R5 in R](https://findingspress.org/article/21262-r5r-rapid-realistic-routing-on-multimodal-transport-networks-with-r-5-in-r) (Published article with accessibility analysis example using sample data)
  1. [Calculating and visualizing Accessibility](https://ipeagit.github.io/r5r/articles/calculating_accessibility.html) (Vignette with accessibility analysis example using sample data)
  1. [osmextract project website](https://docs.ropensci.org/osmextract/)

https://anderfernandez.com/en/blog/how-to-program-with-python-and-r-in-the-same-jupyter-notebook/

https://medium.com/@Ben_Obe/introduction-to-presenting-with-juypter-with-reveal-js-8e34a07081b2

https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/make-maps-with-ggplot-in-R/

## Appendix: Download OpenStreetMap database

In [50]:
start_time = Sys.time()
oe_download(
  file_url = "https://download.geofabrik.de/north-america/us/ohio-latest.osm.pbf", 
  download_directory = "./input_data"
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))

  |======================================================================| 100%


File downloaded!



[1] "C:\\Users\\aporr\\Desktop\\Work\\2022 Ohio GIS Conference\\input_data\\geofabrik_ohio-latest.osm.pbf"

In [52]:
region <- st_read("./temp_data/region.shp")
region <- st_transform(region, crs="epsg:4326")

Reading layer `region' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\region.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 12 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -84.01476 ymin: 39.16777 xmax: -82.02422 ymax: 40.71253
Geodetic CRS:  NAD83


In [55]:
# Use this if you want to create a GeoPackage that includes the entire GeoFabrik dataset (see below for ano
start_time = Sys.time()
oe_vectortranslate("./input_data/geofabrik_ohio-latest.osm.pbf", boundary=region)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))

Start with the vectortranslate operations on the input file!



0...10...20...30...40...50...60...70...80...90...100 - done.


Finished the vectortranslate operations on the input file!



[1] "./input_data/geofabrik_ohio-latest.gpkg"

[1] "Compute time: 0.403892346223195 min"
